# This case study consists of 1000 marks and the solution file is expected to be submitted in the upcoming session for evaluation.

In [ ]:
Data wrangling / Data munging

Data wrangling , sometimes referred to as data munging, is the process of transforming and mapping data from one "raw" data form into another format with the intent of making it more appropriate and valuable for a variety of downstream purposes such as analytics.

The data transformations are typically applied to distinct entities (e.g. fields, rows, columns, data values etc.) within a data set, and could include such actions as extractions, parsing, joining, standardizing, augmenting, cleansing, consolidating and filtering to create desired wrangling outputs that can be leveraged downstream.

The recipients could be individuals, such as data architects or data scientists who will investigate the data further, business users who will consume the data directly in reports, or systems that will further process the data and write it into targets such as data warehouses, data lakes or downstream applications.

Data munging broadly consists of following steps :

* Data Ingestion
* Data merging
* Quick data exploration
* Checking initial stats of the data
* Cleaning the data 
    * check for missing values
    * Treat missing values
    * check for inconsistencies in names / values across data columns
    * Treat data inconsistencies
    * renaming columns names
    * introduce new columns after cleaning
* Exploratory Data Analysis (EDA)
    * Data visualisation
    * Data distribution checking
    * Detailed data stats study
    * Check and treat outliers
* Feature engineering 
    * Binning
    * categorical features to one hot feature conversion
    * categorical features to factorized features
    * creating interactive features
    * creating polynomial features
    * deriving features
    * acquiring more features
* Preprocesssing 
    * feature scaling
    * feature normalization 
    * training data format changing as per ML algo 

## Import libraries

In [1]:
import pandas as pd
import numpy as np

## Data input

In [2]:
# Download
DATASET = (
    "http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    "http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names",
    "http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
)

In [3]:
# Load Training and Test Data Sets
headers = ['age', 'workclass', 'fnlwgt', 
           'education', 'education-num', 
           'marital-status', 'occupation', 
           'relationship', 'race', 'sex', 
           'capital-gain', 'capital-loss', 
           'hours-per-week', 'native-country', 
           'predclass']
training_raw = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', 
                       header=None, 
                       names=headers,na_values=[" ?"])
test_raw = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test', 
                      header=None, 
                      names=headers,na_values=[" ?"])

In [4]:
# Join Datasets
dataset_raw = training_raw.append(test_raw)
dataset_raw.reset_index(inplace=True)
dataset_raw.drop('index',inplace=True,axis=1)

## Data intial stats checking

In [81]:
dataset_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48843 entries, 0 to 48842
Data columns (total 15 columns):
age               48843 non-null object
workclass         46043 non-null object
fnlwgt            48842 non-null float64
education         48842 non-null object
education-num     48842 non-null float64
marital-status    48842 non-null object
occupation        46033 non-null object
relationship      48842 non-null object
race              48842 non-null object
sex               48842 non-null object
capital-gain      48842 non-null float64
capital-loss      48842 non-null float64
hours-per-week    48842 non-null float64
native-country    47985 non-null object
predclass         48842 non-null object
dtypes: float64(5), object(10)
memory usage: 5.6+ MB


In [82]:
# Describing all the Numerical Features
dataset_raw.describe()

fnlwgt  education-num  capital-gain  capital-loss  hours-per-week
count  4.884200e+04   48842.000000  48842.000000  48842.000000    48842.000000
mean   1.896641e+05      10.078089   1079.067626     87.502314       40.422382
std    1.056040e+05       2.570973   7452.019058    403.004552       12.391444
min    1.228500e+04       1.000000      0.000000      0.000000        1.000000
25%    1.175505e+05       9.000000      0.000000      0.000000       40.000000
50%    1.781445e+05      10.000000      0.000000      0.000000       40.000000
75%    2.376420e+05      12.000000      0.000000      0.000000       45.000000
max    1.490400e+06      16.000000  99999.000000   4356.000000       99.000000

In [83]:
# Describing all the Categorical Features
# dataset_raw.describe(include=['O'])
dataset_raw.select_dtypes(include=['object']).describe()

age workclass education       marital-status       occupation  \
count   48843     46043     48842                48842            46033   
unique    147         8        16                    7               14   
top        36   Private   HS-grad   Married-civ-spouse   Prof-specialty   
freq      898     33906     15784                22379             6172   

       relationship    race    sex  native-country predclass  
count         48842   48842  48842           47985     48842  
unique            6       5      2              41         4  
top         Husband   White   Male   United-States     <=50K  
freq          19716   41762  32650           43832     24720

## Q1) Check for missing values

In [84]:
dataset_raw.columns[dataset_raw.isnull().any()]

Index(['workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
       'occupation', 'relationship', 'race', 'sex', 'capital-gain',
       'capital-loss', 'hours-per-week', 'native-country', 'predclass'],
      dtype='object')

## Q2) For columns with missing values, print the number of rows with missing values

In [85]:
dataset_raw.isnull().sum()

age                  0
workclass         2800
fnlwgt               1
education            1
education-num        1
marital-status       1
occupation        2810
relationship         1
race                 1
sex                  1
capital-gain         1
capital-loss         1
hours-per-week       1
native-country     858
predclass            1
dtype: int64

## Q3) For columns with missing values, print the percentage of rows with missing values

In [86]:
missing_values = dataset_raw.isnull().sum()
missing_values/len(dataset_raw)

age               0.000000
workclass         0.057327
fnlwgt            0.000020
education         0.000020
education-num     0.000020
marital-status    0.000020
occupation        0.057531
relationship      0.000020
race              0.000020
sex               0.000020
capital-gain      0.000020
capital-loss      0.000020
hours-per-week    0.000020
native-country    0.017566
predclass         0.000020
dtype: float64

In [87]:
# Another method to achieve the same. COunt does not account for missing values.
1 - dataset_raw.count()/len(dataset_raw)

age               0.000000
workclass         0.057327
fnlwgt            0.000020
education         0.000020
education-num     0.000020
marital-status    0.000020
occupation        0.057531
relationship      0.000020
race              0.000020
sex               0.000020
capital-gain      0.000020
capital-loss      0.000020
hours-per-week    0.000020
native-country    0.017566
predclass         0.000020
dtype: float64

In [88]:
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

## Q4) Use the DataFrameImputer class defined above to impute values to rows with  missing values.

In [89]:
# Create an object of the above
imputer = DataFrameImputer()
# fit the dataset in the above dataset
imputer = imputer.fit(dataset_raw)
# Transform the dataset as well
imputer.transform(dataset_raw)

age          workclass    fnlwgt      education  education-num  \
0      39          State-gov   77516.0      Bachelors           13.0   
1      50   Self-emp-not-inc   83311.0      Bachelors           13.0   
2      38            Private  215646.0        HS-grad            9.0   
3      53            Private  234721.0           11th            7.0   
4      28            Private  338409.0      Bachelors           13.0   
5      37            Private  284582.0        Masters           14.0   
6      49            Private  160187.0            9th            5.0   
7      52   Self-emp-not-inc  209642.0        HS-grad            9.0   
8      31            Private   45781.0        Masters           14.0   
9      42            Private  159449.0      Bachelors           13.0   
10     37            Private  280464.0   Some-college           10.0   
11     30          State-gov  141297.0      Bachelors           13.0   
12     23            Private  122272.0      Bachelors           13.0   
13     32            Private  205019.0     Assoc-acdm           12.0   
14     40            Private  121772.0      Assoc-voc           11.0   
15     34            Private  245487.0        7th-8th            4.0   
16     25   Self-emp-not-inc  176756.0        HS-grad            9.0   
17     32            Private  186824.0        HS-grad            9.0   
18     38            Private   28887.0           11th            7.0   
19     43   Self-emp-not-inc  292175.0        Masters           14.0   
20     40            Private  193524.0      Doctorate           16.0   
21     54            Private  302146.0        HS-grad            9.0   
22     35        Federal-gov   76845.0            9th            5.0   
23     43            Private  117037.0           11th            7.0   
24     59            Private  109015.0        HS-grad            9.0   
25     56          Local-gov  216851.0      Bachelors           13.0   
26     19            Private  168294.0        HS-grad            9.0   
27     54            Private  180211.0   Some-college           10.0   
28     39            Private  367260.0        HS-grad            9.0   
29     49            Private  193366.0        HS-grad            9.0   
...    ..                ...       ...            ...            ...   
48813  81            Private   26711.0      Assoc-voc           11.0   
48814  60            Private  117909.0      Assoc-voc           11.0   
48815  39            Private  229647.0      Bachelors           13.0   
48816  38            Private  149347.0        Masters           14.0   
48817  43          Local-gov   23157.0        Masters           14.0   
48818  23            Private   93977.0        HS-grad            9.0   
48819  73       Self-emp-inc  159691.0   Some-college           10.0   
48820  35            Private  176967.0   Some-college           10.0   
48821  66            Private  344436.0        HS-grad            9.0   
48822  27            Private  430340.0   Some-college           10.0   
48823  40            Private  202168.0    Prof-school           15.0   
48824  51            Private   82720.0        HS-grad            9.0   
48825  22            Private  269623.0   Some-college           10.0   
48826  64   Self-emp-not-inc  136405.0        HS-grad            9.0   
48827  50          Local-gov  139347.0        Masters           14.0   
48828  55            Private  224655.0        HS-grad            9.0   
48829  38            Private  247547.0      Assoc-voc           11.0   
48830  58            Private  292710.0     Assoc-acdm           12.0   
48831  32            Private  173449.0        HS-grad            9.0   
48832  48            Private  285570.0        HS-grad            9.0   
48833  61            Private   89686.0        HS-grad            9.0   
48834  31            Private  440129.0        HS-grad            9.0   
48835  25            Private  350977.0        HS-grad            9.0   
48836  48          Local-gov  349230.0        

## Q5) Use appropriate formatting for all columns

In [28]:
dataset_raw['occupation'].unique()
dataset_raw['occupation'] = dataset_raw['occupation'].apply(lambda x : str(x).strip())

In [5]:
# Remove the leading spaces in all the obejct columns. Categorical variables
for i in dataset_raw.describe(include=['O']).columns:
    dataset_raw[i] = dataset_raw[i].apply(lambda x : str(x).strip(" "))